In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math

In [14]:
df = pd.read_csv("apple.csv")
df.set_index(df["Date"], inplace = True)
df.head()
df.drop("Date", axis = 1, inplace = True)
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,17633200.0,0.0,1.0,27.428730,27.566240,27.293790,27.503268,123432400.0
2010-01-05,214.60,215.59,213.25,214.38,21496600.0,0.0,1.0,27.579091,27.706320,27.405597,27.550818,150476200.0
2010-01-06,214.38,215.23,210.75,210.97,19720000.0,0.0,1.0,27.550818,27.660055,27.084312,27.112585,138040000.0
2010-01-07,211.75,212.00,209.05,210.58,17040400.0,0.0,1.0,27.212826,27.244955,26.865839,27.062465,119282800.0
2010-01-08,210.30,212.00,209.06,211.98,15986100.0,0.0,1.0,27.026481,27.244955,26.867124,27.242385,111902700.0


In [15]:

df["HL_PCT"] = ((df["Adj. High"] - df["Adj. Low"]) / df["Adj. Low"])*100
df["PCT_change"] = ((df["Adj. Close"] - df["Adj. Open"]) / df["Adj. Open"])*100

df = df[["Adj. Close", "HL_PCT", "PCT_change", "Adj. Volume"]]

df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2010-01-04,27.503268,0.998211,0.271752,123432400.0
2010-01-05,27.550818,1.097304,-0.102516,150476200.0
2010-01-06,27.112585,2.125741,-1.590633,138040000.0
2010-01-07,27.062465,1.411146,-0.552538,119282800.0
2010-01-08,27.242385,1.406295,0.798859,111902700.0


In [16]:
to_predict = "Adj. Close"
df.isnull().any().any()
number_predictions = 1
number_days_into_account = 1
df["Predicted"] = df[to_predict].shift(-number_days_into_account)
df.head(7)
df.dropna(inplace = True)
df.head()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,Predicted
Date,,,,,
2010-01-04,27.503268,0.998211,0.271752,123432400.0,27.550818
2010-01-05,27.550818,1.097304,-0.102516,150476200.0,27.112585
2010-01-06,27.112585,2.125741,-1.590633,138040000.0,27.062465
2010-01-07,27.062465,1.411146,-0.552538,119282800.0,27.242385
2010-01-08,27.242385,1.406295,0.798859,111902700.0,27.002063


In [17]:
train = df.iloc[0:int(math.ceil(0.8*len(df)))]
test = df.iloc[int(math.ceil(0.8*len(df)))::]
train.columns = ["AdjClose","HL_PCT","PCT_change","AdjVolume","Predicted"]
test.columns = ["AdjClose","HL_PCT","PCT_change","AdjVolume","Predicted"]

In [21]:
comparar = test["Predicted"]

In [22]:
formula = "Predicted ~ AdjClose + HL_PCT + PCT_change + AdjVolume"
model = smf.ols(formula, data = train)
result = model.fit()

In [23]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Predicted   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.928e+05
Date:                Tue, 11 Feb 2020   Prob (F-statistic):               0.00
Time:                        19:26:32   Log-Likelihood:                -2176.4
No. Observations:                1409   AIC:                             4363.
Df Residuals:                    1404   BIC:                             4389.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0999      0.135      0.741      0.459      -0.164       0.364
AdjClose       0.9995      0.001    736.684      0.000       0.997       1.002
HL_PCT        -0.0040      0.031     -0.128      0.898      -0.065       0.057
PCT_change     0.0025      0.023      0.108      0.914      -0.043       0.048
AdjVolume   3.632e-11   7.48e-10      0.049      0.961   -1.43e-09     1.5e-09
==============================================================================
Omnibus:                      180.162   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1764.505
Skew:                          -0.166   Prob(JB):                         0.00
Kurtosis:                       8.472   Cond. No.                     5.47e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.47e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
test.drop("Predicted", axis = 1, inplace = True)

In [25]:
test.head()

,AdjClose,HL_PCT,PCT_change,AdjVolume
Date,,,,
2015-08-10,115.172038,2.969192,2.737492,54951597.0
2015-08-11,109.178705,4.279538,-3.666921,97082814.0
2015-08-12,110.862226,5.281401,2.408247,101685610.0
2015-08-13,110.775645,1.623887,-0.766977,48535789.0
2015-08-14,111.554874,2.017367,1.434570,42929516.0


In [26]:
result.predict(test)-comparar

Date
2015-08-10    6.032372
2015-08-11   -1.661012
2015-08-12    0.119495
2015-08-13   -0.741565
2015-08-14   -1.113448
                ...   
2016-12-22   -0.187128
2016-12-23   -0.689458
2016-12-27    0.533317
2016-12-28    0.064547
2016-12-29    0.939462
Length: 352, dtype: float64

In [12]:
result.predict(test)

Date
2015-08-10    115.211078
2015-08-11    109.201213
2015-08-12    110.895140
2015-08-13    110.813309
2015-08-14    111.595840
                 ...    
2016-12-22    114.893680
2016-12-23    115.122210
2016-12-27    115.851160
2016-12-28    115.352761
2016-12-29    115.328916
Length: 352, dtype: float64

In [26]:
df.tail(100)


,Adj. Close,HL_PCT,PCT_change,Adj. Volume,Predicted
Date,,,,,
2016-08-09,106.911186,0.861031,0.535896,26315204.0,106.115321
2016-08-10,106.115321,1.057906,-0.653114,24008505.0,106.046542
2016-08-11,106.046542,1.001391,-0.543679,27484506.0,106.292180
2016-08-12,106.292180,0.612359,0.371126,18660434.0,107.569494
2016-08-15,107.569494,1.350851,1.239134,25868209.0,107.471239
2016-08-16,107.471239,0.933980,-0.228040,33794448.0,107.314031
2016-08-17,107.314031,0.950711,0.109991,25355976.0,107.176474
2016-08-18,107.176474,0.532012,-0.137325,21984703.0,107.451588
2016-08-19,107.451588,1.227390,0.542429,25368072.0,106.616421


In [27]:
#VERSION MEJORADA

In [105]:
df = pd.read_csv("apple.csv")
df.set_index("Date", inplace = True)
df["HL_PCT"] = ((df["Adj. High"] - df["Adj. Low"]) / df["Adj. Low"])*100
df["PCT_change"] = ((df["Adj. Close"] - df["Adj. Open"]) / df["Adj. Open"])*100
df = df[["Adj. Close", "HL_PCT", "PCT_change", "Adj. Volume"]]
df.columns = ["AdjClose","HL_PCT", "PCT_change", "AdjVolume"]
df.head()

,AdjClose,HL_PCT,PCT_change,AdjVolume
Date,,,,
2010-01-04,27.503268,0.998211,0.271752,123432400.0
2010-01-05,27.550818,1.097304,-0.102516,150476200.0
2010-01-06,27.112585,2.125741,-1.590633,138040000.0
2010-01-07,27.062465,1.411146,-0.552538,119282800.0
2010-01-08,27.242385,1.406295,0.798859,111902700.0


In [106]:
past_days = 5
columnas = df.columns.tolist()

In [107]:
for i in range(past_days):
    for j in columnas:
        df[j+str(i)] = np.nan

In [108]:
nuevas_columnas = df.columns.tolist()
columnas
del nuevas_columnas[0:4]

In [109]:
for i in nuevas_columnas:
    df[i] = df[i[:-1]].shift((int(i[-1:])+1))

In [110]:
df.dropna(inplace = True)

In [111]:
predictors = df.columns
predictors = predictors.tolist()
del predictors[0:4]

In [112]:
df2 = df.copy()
columnas_datos = df.columns.tolist()
del columnas_datos[1:4]
df2 = df2[columnas_datos]

In [114]:
train = df2.iloc[0:int(math.ceil(0.8*len(df2)))]
test = df2.iloc[int(math.ceil(0.8*len(df2)))::]

In [117]:
formula = "AdjClose ~ AdjClose0"
for i in columnas_datos[2::]:
    formula = formula +"+"+i

model = smf.ols(formula, data = train)
result = model.fit()

In [118]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               AdjClose   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 3.765e+04
Date:                Tue, 31 Dec 2019   Prob (F-statistic):               0.00
Time:                        16:51:29   Log-Likelihood:                -2177.2
No. Observations:                1406   AIC:                             4396.
Df Residuals:                    1385   BIC:                             4507.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.1839      0.161      1.140      0.255      -0.133       0.501
AdjClose0       1.0170      0.039     26.279      0.000       0.941       1.093
HL_PCT0        -0.0369      0.033     -1.111      0.267      -0.102       0.028
PCT_change0    -0.0101      0.033     -0.305      0.760      -0.075       0.055
AdjVolume0   9.061e-10   9.81e-10      0.924      0.356   -1.02e-09    2.83e-09
AdjClose1      -0.0861      0.054     -1.590      0.112      -0.192       0.020
HL_PCT1         0.0531      0.034      1.578      0.115      -0.013       0.119
PCT_change1     0.0313      0.033      0.941      0.347      -0.034       0.097
AdjVolume1  -1.152e-09    1.1e-09     -1.052      0.293    -3.3e-09    9.97e-10
AdjClose2       0.0700      0.054      1.286      0.199      -0.037       0.177
HL_PCT2         0.0395      0.034      1.177      0.239      -0.026       0.105
PCT_change2    -0.0084      0.033     -0.253      0.801      -0.074       0.057
AdjVolume2  -9.537e-10    1.1e-09     -0.870      0.384    -3.1e-09     1.2e-09
AdjClose3       0.0321      0.054      0.590      0.555      -0.075       0.139
HL_PCT3         0.0582      0.034      1.735      0.083      -0.008       0.124
PCT_change3     0.0279      0.033      0.838      0.402      -0.037       0.093
AdjVolume3   -5.41e-12    1.1e-09     -0.005      0.996   -2.16e-09    2.15e-09
AdjClose4      -0.0351      0.039     -0.899      0.369      -0.112       0.041
HL_PCT4        -0.0230      0.033     -0.702      0.483      -0.087       0.041
PCT_change4    -0.0179      0.024     -0.753      0.452      -0.065       0.029
AdjVolume4  -3.534e-10   9.69e-10     -0.365      0.715   -2.25e-09    1.55e-09
==============================================================================
Omnibus:                      192.922   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1906.658
Skew:                          -0.253   Prob(JB):                         0.00
Kurtosis:                       8.682   Cond. No.                     1.40e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [119]:
result.predict(test)

Date
2015-08-12    108.845431
2015-08-13    111.251608
2015-08-14    111.265663
2015-08-17    111.383602
2015-08-18    113.083647
2015-08-19    111.905130
2015-08-20    110.808970
2015-08-21    108.472220
2015-08-24    101.692935
2015-08-25     99.008719
2015-08-26    101.022707
2015-08-27    105.939356
2015-08-28    109.798056
2015-08-31    108.518339
2015-09-01    108.909344
2015-09-02    103.667383
2015-09-03    108.490198
2015-09-04    106.090176
2015-09-08    105.166208
2015-09-09    108.507715
2015-09-10    105.642113
2015-09-11    108.434185
2015-09-14    110.069091
2015-09-15    110.964434
2015-09-16    112.007245
2015-09-17    112.076122
2015-09-18    109.527599
2015-09-21    109.345415
2015-09-22    110.935203
2015-09-23    108.944441
                 ...    
2016-11-17    108.710334
2016-11-18    108.554773
2016-11-21    108.839459
2016-11-22    110.655014
2016-11-23    110.298035
2016-11-25    109.851809
2016-11-28    110.592883
2016-11-29    110.135870
2016-11-30    110.06

In [122]:
test.head(15)

,AdjClose,AdjClose0,HL_PCT0,PCT_change0,AdjVolume0,AdjClose1,HL_PCT1,PCT_change1,AdjVolume1,AdjClose2,...,PCT_change2,AdjVolume2,AdjClose3,HL_PCT3,PCT_change3,AdjVolume3,AdjClose4,HL_PCT4,PCT_change4,AdjVolume4
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-12,110.862226,109.178705,4.279538,-3.666921,97082814.0,115.172038,2.969192,2.737492,54951597.0,111.131589,...,0.820388,38670405.0,110.756405,2.089997,-0.724325,52903040.0,110.516983,4.763604,2.169101,98384461.0
2015-08-13,110.775645,110.862226,5.281401,2.408247,101685610.0,109.178705,4.279538,-3.666921,97082814.0,115.172038,...,2.737492,54951597.0,111.131589,1.528384,0.820388,38670405.0,110.756405,2.089997,-0.724325,52903040.0
2015-08-14,111.554874,110.775645,1.623887,-0.766977,48535789.0,110.862226,5.281401,2.408247,101685610.0,109.178705,...,-3.666921,97082814.0,115.172038,2.969192,2.737492,54951597.0,111.131589,1.528384,0.820388,38670405.0
2015-08-17,112.709288,111.554874,2.017367,1.434570,42929516.0,110.775645,1.623887,-0.766977,48535789.0,110.862226,...,2.408247,101685610.0,109.178705,4.279538,-3.666921,97082814.0,115.172038,2.969192,2.737492,54951597.0
2015-08-18,112.074361,112.709288,1.861472,0.965184,40884745.0,111.554874,2.017367,1.434570,42929516.0,110.775645,...,-0.766977,48535789.0,110.862226,5.281401,2.408247,101685610.0,109.178705,4.279538,-3.666921,97082814.0
2015-08-19,110.640963,112.074361,1.232652,0.060122,34560708.0,112.709288,1.861472,0.965184,40884745.0,111.554874,...,1.434570,42929516.0,110.775645,1.623887,-0.766977,48535789.0,110.862226,5.281401,2.408247,101685610.0
2015-08-20,108.370616,110.640963,1.604465,-0.938846,48286510.0,112.074361,1.232652,0.060122,34560708.0,112.709288,...,0.965184,40884745.0,111.554874,2.017367,1.434570,42929516.0,110.775645,1.623887,-0.766977,48535789.0
2015-08-21,101.742355,108.370616,2.436621,-1.253506,68501622.0,110.640963,1.604465,-0.938846,48286510.0,112.074361,...,0.060122,34560708.0,112.709288,1.861472,0.965184,40884745.0,111.554874,2.017367,1.434570,42929516.0
2015-08-24,99.202644,101.742355,5.920772,-4.228923,128275471.0,108.370616,2.436621,-1.253506,68501622.0,110.640963,...,-0.938846,48286510.0,112.074361,1.232652,0.060122,34560708.0,112.709288,1.861472,0.965184,40884745.0
